In [10]:
import selenium
from openai import OpenAI
import os
import dotenv

dotenv.load_dotenv()




True

RAG system for selenium agent| storing in vector database

In [5]:
import fitz
print("PyMuPDF is working correctly!")


PyMuPDF is working correctly!


In [7]:
import fitz  # PyMuPDF
import tiktoken

pdf_path = "selenium_documentation.pdf"

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n\n"
    return text

selenium_text = extract_text_from_pdf(pdf_path)

# Split text into smaller chunks for embedding
def chunk_text(text, max_tokens=500):
    tokenizer = tiktoken.get_encoding("cl100k_base")
    words = text.split()
    chunks, current_chunk = [], []

    for word in words:
        current_chunk.append(word)
        if len(tokenizer.encode(" ".join(current_chunk))) >= max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = []

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

text_chunks = chunk_text(selenium_text)
print(f"Generated {len(text_chunks)} chunks.")


Generated 402 chunks.


In [9]:
import chromadb


# Initialize OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(name="selenium_docs")

# Embed and store chunks
for i, chunk in enumerate(text_chunks):
    response = client.embeddings.create(input=[chunk], model="text-embedding-3-small")  # Updated API
    embedding = response.data[0].embedding  # Corrected attribute access
    collection.add(documents=[chunk], embeddings=[embedding], ids=[str(i)])

print("Stored Selenium docs in the vector database.")


Stored Selenium docs in the vector database.


In [21]:
def query_selenium_docs(query):
    # Initialize OpenAI client
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    # Generate query embedding
    response = client.embeddings.create(input=[query], model="text-embedding-3-small")
    query_embedding = response.data[0].embedding

    # Retrieve top 3 relevant chunks
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=3
    )

    # Combine retrieved documents
    if results and results.get("documents"):
        context = " ".join(results["documents"][0])  # Combine top results
    else:
        context = "No relevant information found in the documentation."
    print("Context: ", context)
    # Generate final response using GPT-4o (corrected model)
    prompt = f"Use the following Selenium documentation to answer the query:\n\n{context}\n\nUser query: {query}\n\nResponse:"
    
    completion = client.chat.completions.create(
        model="gpt-4o",  # ✅ Use GPT-4o (correct model)
        messages=[
            {"role": "user", "content": prompt}  # ✅ Removed "system" role (unsupported)
        ]
    )

    return completion.choices[0].message.content  # ✅ Corrected attribute access

Test Retrival System

In [22]:
#test

query = '''Create a test to check if this local website is responsive here is the code: 
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Responsive Like Buttons</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            background-color: #f4f4f4;
            margin: 0;
        }
        .container {
            width: 80%;
            max-width: 600px;
            background: white;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
            text-align: center;
        }
        .like-button {
            background-color: #007bff;
            color: white;
            border: none;
            padding: 10px 20px;
            font-size: 16px;
            cursor: pointer;
            border-radius: 5px;
            transition: background 0.3s;
        }
        .like-button:hover {
            background-color: #0056b3;
        }
        .count {
            font-size: 18px;
            margin-top: 10px;
        }
    </style>
</head>
<body>
    <div class="container">
        <h2>Like This Post</h2>
        <button class="like-button" onclick="likePost()">Like</button>
        <p class="count">Likes: <span id="like-count">0</span></p>
    </div>

    <script>
        let likeCount = 0;
        function likePost() {
            likeCount++;
            document.getElementById("like-count").textContent = likeCount;
        }
    </script>
</body>
</html>


'''
print(query_selenium_docs(query))

Context:  } </script> </head> <body> <input type="button" id="btnConfirm" onclick="show_confirm()" value="Show confir 3/1/25, 10:28 AM The Selenium Browser Automation Project | Selenium https://www.selenium.dev/documentation/_print/ 521/698 The user must respond to alert/confirm boxes, as well as moving focus to newly opened popup windows. Fortunately, Selenium can cover JavaScript pop-ups. But before we begin covering alerts/confirms/prompts in individual detail, it is helpful to understand the commonality between them. Alerts, confirmation boxes and prompts all have variations of the following Command Description assertFoo(pattern) throws error if pattern doesn’t match the text of the pop-up assertFooPresent throws error if pop-up is not available assertFooNotPresent throws error if any pop-up is present storeFoo(variable) stores the text of the pop-up in a variable storeFooPresent(variable) stores the text of the pop-up in a variable and returns true or false When running under Sele

In [ ]:
# step 1: user asks to make website
user_input = 'make me a website that has a red button that says "Click me" and when clicked it changes the buttons background color to blue and if clicked again it changes it back to red.'
# step 2: webdev agent creates website then asks selenium agent to test it using selenium

# step 3: selenium agent identifies key testing points and creates tests for them, then runs them and gives feedback to the webdev agent.
# step 4: webdev agent fixes the issues and the cycle continues until the website is perfect.
